# 0. Import Libraries

In [9]:
from __future__ import print_function
import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os

# 1. Read the the dataset

In [10]:
num_classes=2
img_rows,img_cols=48,48
batch_size=64

train_data_dir='data_set/train'
validation_data_dir='data_set/validation'

train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					width_shift_range=0.4,
					height_shift_range=0.4,
					horizontal_flip=True,
					fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)
					
validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)

model = Sequential()


Found 71 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


# 2. Fit the model

In [11]:
#Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-4 

# model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.2))

#Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

#Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_18 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_19 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 24, 24, 32)       

# 3. Train the model

In [12]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from time import time

# checkPointName = 'EmotionDetectionModel.h5'
#quick fix to increment the file if it exists
counter = 0
filename = "EmotionDetectionModel{}.h5"
while os.path.isfile(filename.format(counter)):
    counter += 1
checkPointName = filename.format(counter)
#quick fix to increment the file if it exists

checkpoint = ModelCheckpoint(checkPointName,
                             monitor='val_loss',
                             mode='min',
                             verbose=1)

earlystop = EarlyStopping(monitor='accuracy',
                          min_delta=0,
                          patience=5000,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='accuracy',
                              factor=0.2,
                              patience=5000,
                              verbose=1,
                              min_delta=0.0001)

# Create a TensorBoard instance with the path to the logs directory
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))

callbacks = [earlystop,checkpoint,reduce_lr, tensorboard]
''', tensorboard'''

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 71
nb_validation_samples = 22
epochs=100

history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

Epoch 1/100
2/2 [==============================] - 3s 1s/step - loss: 1.4966 - accuracy: 0.4359

Epoch 00001: saving model to EmotionDetectionModel2.h5
Epoch 2/100
2/2 [==============================] - 1s 392ms/step - loss: 1.0058 - accuracy: 0.6094

Epoch 00002: saving model to EmotionDetectionModel2.h5
Epoch 3/100
2/2 [==============================] - 1s 413ms/step - loss: 0.9082 - accuracy: 0.6410

Epoch 00003: saving model to EmotionDetectionModel2.h5
Epoch 4/100
2/2 [==============================] - 1s 139ms/step - loss: 0.5714 - accuracy: 0.7179

Epoch 00004: saving model to EmotionDetectionModel2.h5
Epoch 5/100
2/2 [==============================] - 0s 377ms/step - loss: 1.3189 - accuracy: 0.5641

Epoch 00005: saving model to EmotionDetectionModel2.h5
Epoch 6/100
2/2 [==============================] - 1s 130ms/step - loss: 0.9022 - accuracy: 0.7436

Epoch 00006: saving model to EmotionDetectionModel2.h5
Epoch 7/100
2/2 [==============================] - 0s 355ms/step - loss: 